In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Dataset
import itertools

In [2]:
train = pd.read_csv('yelp/train_en.txt', sep='\t')[['Sentence', 'Style']]
val   = pd.read_csv('yelp/val_en.txt', sep='\t')[['Sentence', 'Style']]
test  = pd.read_csv('yelp/test_en.txt', sep='\t')[['Sentence', 'Style']]

In [3]:
train_small = train.sample(5000, random_state=42)
val_small   = val.sample(2000, random_state=42)
test_small  = test.sample(2000, random_state=42)

In [4]:
label_map = {'positive': 1, 'negative': 0}
for df in [train_small, val_small, test_small]:
    df['Label'] = df['Style'].map(label_map)

In [5]:
class YelpDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len=128):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = str(self.sentences[idx])
        encoding = self.tokenizer(sentence, 
                                  padding='max_length', 
                                  truncation=True,
                                  max_length=self.max_len,
                                  return_tensors='pt')
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [6]:
def train_evaluate_transformer(model_name, train_df, val_df, test_df, epochs=1, lr=5e-5):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    train_dataset = YelpDataset(train_df['Sentence'].tolist(), train_df['Label'].tolist(), tokenizer)
    val_dataset   = YelpDataset(val_df['Sentence'].tolist(), val_df['Label'].tolist(), tokenizer)
    test_dataset  = YelpDataset(test_df['Sentence'].tolist(), test_df['Label'].tolist(), tokenizer)

    training_args = TrainingArguments(
        output_dir=f'{model_name}_results',
        eval_strategy="epoch",
        learning_rate=lr,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=epochs,
        weight_decay=0.01,
        save_strategy="no",
        logging_strategy="epoch",
        report_to="none"
    )

    def compute_metrics(p):
        preds = p.predictions.argmax(-1)
        labels = p.label_ids
        return {
            'accuracy': accuracy_score(labels, preds),
            'precision': precision_score(labels, preds),
            'recall': recall_score(labels, preds),
            'f1': f1_score(labels, preds)
        }

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()
    results = trainer.predict(test_dataset)
    metrics = {
        'accuracy': accuracy_score(test_df['Label'], results.predictions.argmax(-1)),
        'precision': precision_score(test_df['Label'], results.predictions.argmax(-1)),
        'recall': recall_score(test_df['Label'], results.predictions.argmax(-1)),
        'f1': f1_score(test_df['Label'], results.predictions.argmax(-1))
    }
    print(f"\nResults for {model_name} (lr={lr}, epochs={epochs}): {metrics}")
    return metrics

In [7]:
model_names = ["roberta-base", "distilbert-base-uncased"]
lr_list = [5e-5, 3e-5, 1e-5]
epoch_list = [2, 3]

In [8]:
all_results = []
for model_name, lr, epochs in itertools.product(model_names, lr_list, epoch_list):
    res = train_evaluate_transformer(model_name, train_small, val_small, test_small, epochs=epochs, lr=lr)
    all_results.append({
        'model': model_name,
        'lr': lr,
        'epochs': epochs,
        **res
    })

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.430500,0.403898,0.860000,0.907746,0.896384,0.902029
2,0.290900,0.456899,0.875000,0.900270,0.929068,0.914442



Results for roberta-base (lr=5e-05, epochs=2): {'accuracy': 0.8925, 'precision': 0.9109176155391828, 'recall': 0.9431345353675451, 'f1': 0.9267461669505963}


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.434300,0.494787,0.862000,0.932292,0.871349,0.900791
2,0.309700,0.422773,0.875500,0.912561,0.914465,0.913512
3,0.191400,0.507701,0.883000,0.911202,0.927677,0.919366


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.388100,0.371343,0.857500,0.894053,0.909597,0.901758
2,0.289400,0.436086,0.862500,0.882818,0.932545,0.907000



Results for distilbert-base-uncased (lr=1e-05, epochs=2): {'accuracy': 0.876, 'precision': 0.9006711409395973, 'recall': 0.9306518723994452, 'f1': 0.9154160982264665}


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.400300,0.370751,0.856000,0.900976,0.898470,0.899721
2,0.291800,0.399385,0.864500,0.898294,0.915160,0.906648
3,0.231500,0.474237,0.863000,0.886968,0.927677,0.906866



Results for distilbert-base-uncased (lr=1e-05, epochs=3): {'accuracy': 0.8765, 'precision': 0.9007377598926894, 'recall': 0.9313453536754508, 'f1': 0.9157858847596317}


In [9]:
results_df = pd.DataFrame(all_results)
print(results_df)

                      model       lr  epochs  accuracy  precision    recall  \
0              roberta-base  0.00005       2    0.8925   0.910918  0.943135   
1              roberta-base  0.00005       3    0.8935   0.913248  0.941748   
2              roberta-base  0.00003       2    0.9005   0.911862  0.954230   
3              roberta-base  0.00003       3    0.9000   0.925342  0.936893   
4              roberta-base  0.00001       2    0.8970   0.901299  0.962552   
5              roberta-base  0.00001       3    0.9055   0.919064  0.952843   
6   distilbert-base-uncased  0.00005       2    0.8745   0.900471  0.928571   
7   distilbert-base-uncased  0.00005       3    0.8695   0.898179  0.923717   
8   distilbert-base-uncased  0.00003       2    0.8750   0.899464  0.930652   
9   distilbert-base-uncased  0.00003       3    0.8740   0.904762  0.922330   
10  distilbert-base-uncased  0.00001       2    0.8760   0.900671  0.930652   
11  distilbert-base-uncased  0.00001       3    0.87

In [22]:
# Со менување на хиперпараметрите, перформансите на моделот минимално се менуваа
# Резултатите споредени со моделите од првата лабараториска вежба се скоро исти